In [ ]:
# Esse arquivo foi feito para ser executado no Google Colab
!git clone https://github.com/lucas-wa/angelo.git
!mv angelo/* .
!rm -rf ./angelo

In [ ]:
!pip install realesrgan

In [ ]:
!pip install basicsr facexlib gfpgan opencv-python Pillow torch torchvision tqdm

In [ ]:
!pip install torch

In [ ]:
# Instalando dependências do ngrock
!pip install flask-ngrok flask-cors
!pip install pyngrok

In [ ]:
!ngrok config add-authtoken YOUR_NGROK_KEY

In [ ]:
# Instalando stable diffusion
!pip install diffusers transformers accelerate scipy safetensors


In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
from diffusers import StableDiffusionInpaintPipeline

editor_pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
)
editor_pipe.to("cuda")

In [ ]:
import os
import io
import base64
import queue
import threading
from PIL import Image

from flask import Flask, render_template, request, jsonify
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from flask_cors import CORS

from inference_realesrgan import upscale_image

# from classes.stable_diffusion import StableDiffusion
# from classes.stable_painting import StablePainting
from classes.esrgan import ESRGAN

request_queue = queue.Queue()

app = Flask(__name__)
cors = CORS(app, resources={r"/*": {"origins": "*"}})
port = 5000



public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

app.config["BASE_URL"] = public_url
app.config['UPLOAD_FOLDER'] = 'images/'


def process_requests():
    while True:
      try:
        req, service, response_queue, status_code = request_queue.get()

        if service == "generator":

          prompt = req["prompt"]
          stable_diffusion = StableDiffusion("", prompt)
          response = stable_diffusion.gerar_imagem()
          status_code = 200
          response_queue.put((response, status_code))

        elif service == "upscale":

          # Process the request here and generate the response
          file = req['file']
          file = base64.b64decode(file)
          filename = "image.png"

          esrgan = ESRGAN("")

          image = esrgan.upscale_imagem(file, "image.png")
          image = Image.fromarray(image)
          buffered = io.BytesIO()
          image.save(buffered, format="PNG")
          img_str = base64.b64encode(buffered.getvalue())
          response = "data:image/png;base64," + str(img_str)[2:-1]
          response_queue.put((response, status_code))


        elif service == "editor":

          prompt = req["prompt"]

          stable_painting = StablePainting("", prompt)

          response = stable_painting.editar_imagem(req)

          response_queue.put((response, status_code))


      except Exception as e:
        print(e)
        response = "Internal server error. Image couldn't be generated"
        status_code = 500
        response_queue.put((response, status_code))


@app.route("/", methods = ['GET', 'POST'])
def index():

  return render_template("index.html")


@app.route("/generator", methods = ['POST'])
def image_generator():
  try:
    status_code = 200
    req = request.get_json()
    prompt = req["prompt"]
    service = 'generator'
    if ("prompt" not in req) or (prompt == ''):
          return jsonify({"error": "Prompt is missing"}), 400

    response_queue = queue.Queue()
    request_queue.put((req, service, response_queue, status_code))
    response, status_code = response_queue.get()
    if(status_code == 500):
        return jsonify({"error": "Internal server error"}), 500
    return jsonify({"image_raw": response})
  except Exception as e:
    print(e)
    return jsonify({"error": e})


@app.route("/upscale", methods = ['POST'])
def upsacle_image():
  try:
    req = request.get_json()

    if 'file' not in req:
      return jsonify({"error": "No file"}), 400

    service = 'upscale'


    status_code = 200
    response_queue = queue.Queue()
    request_queue.put((req, service, response_queue, status_code))
    reponse, status_code = response_queue.get()
    return jsonify({"image_raw": reponse}), 200

  except Exception as e:
    print(e)
    return jsonify({"error": "Internal server error"}), 500


@app.route("/editor", methods = ['POST'])
def edit_image():
  try:
    req = request.get_json()

    if 'file' not in req:
      return jsonify({"error": "No file"}), 400

    if 'mask' not in req:
      return jsonify({"error": "No mask"}), 400

    prompt = req["prompt"]

    if ("prompt" not in req) or (prompt == ''):
      return jsonify({"error": "Prompt or service is missing"}), 400

    service = 'editor'


    status_code = 200
    response_queue = queue.Queue()
    request_queue.put((req, service, response_queue, status_code))
    reponse, status_code = response_queue.get()
    return jsonify({"image_raw": reponse}), 200

  except Exception as e:
    print(e)
    return jsonify({"error": "Internal server error"}), 500

tr_stable = threading.Thread(target=process_requests)
tr_stable.start()

app.run()